In [9]:
from datetime import date
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium import webdriver # driving the actions, links up with the browser
from bs4 import BeautifulSoup
import time
import re
from pick import pick
import progressbar
import os

In [10]:
# change the path

search_webpage = 'https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm'
PATH = r'C:\Users\esthe\Documents\Coding\chromedriver_win32\chromedriver.exe'
#driver = webdriver.Chrome(PATH)
#driver.get(search_webpage)

In [11]:
yes = ["Yes", "Y", "YES", "y", "yes"]
starting_page = "https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
dashboard_url = "http://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
login_url = 'https://cas.uwaterloo.ca/cas/login?service=https://waterlooworks.uwaterloo.ca/waterloo.htm'
output_name = 'WW_postings.csv'
choice = 'Application Deadlines in the next 10 Days'
potential_html1 = None
potential_html2 = None
potential_html3 = None

In [ ]:
{"Work Term Duration:": None, # might need to do some regex filtering
                                         "Job Summary:" : None,
                                         "Job Responsibilities:": None,
                                         "Required Skills:" : None,
                                         "Targeted Degrees and Disciplines:": None,
                                         "Application Documents Required:": None} 

In [17]:
df = pd.DataFrame(columns=['ID', 'Job Title', 'Organization', 'Division', 'Openings', 'Internal Status', 'City', 'Level',
                           'Applications', 'App Deadline', 'Work Term Duration', "Job Summary", "Job Responsibilities", "Required Skills", 
                           "Targeted Degrees and Disciplines", "Application Documents Required"])
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required


In [43]:
df.loc[3] = ["hello"] * 17

In [53]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required


In [ ]:
def main():
    starting_page = "https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"

    output_name = 'WW_postings.csv'
    
    browser = webdriver.Chrome(PATH)
    browser.get(starting_page) # in original script, this was starting page

    input('Are you ready?')

    choice = 'Viewed'
    
    data = get_job_lists(choice, browser, output_name)

    print("Done!")


def get_job_lists(choice, browser, output_name):
    
    print("Getting job lists ...")

    # Navigate to the job listings page then wait for the javascript to load
    browser.find_element_by_link_text(choice).click()

    # Get the HTML of the page and check if this is the last page (next_page_buttons)
    page_html = browser.execute_script("return document.body.innerHTML")
    pattern = r'<a href=".+?" onclick="loadPostingTable(.+?)">\s*»\s*<\/a>'
    next_page_buttons = []
    next_page_buttons = re.findall(pattern, page_html)



    # bar = progressbar.ProgressBar(max_value=self.job_lists_page_count)
    # bar.update(self.gather_current_progress)


    # dots for loading screen
    dots = ""

    # Scrape the tables and save it to output_name as a CSV
    job_posting_importants_order = ['Work Term Duration:', "Job Summary:", "Job Responsibilities:", "Required Skills:", 
                                    "Targeted Degrees and Disciplines:", "Application Documents Required:"]
    with open(output_name, 'w') as f:
        row_count = 0
        while (next_page_buttons):
            
            # loading screen stuff
            os.system('clear')
            print("Working" + dots)
            # update dots
            if (dots == "..."):
                dots = ""
            else:
                dots += "."

            # wait for JavaScript to load then download the page info
            time.sleep(2)
            soup = BeautifulSoup(page_html, "html.parser")
            for tr in soup.find_all('tr')[2:]: # iterating throw all rows
                
                # all the data that was not in the original script
                job_posting_importants = {"Work Term Duration:": None, # might need to do some regex filtering
                                         "Job Summary:" : None,
                                         "Job Responsibilities:": None,
                                         "Required Skills:" : None,
                                         "Targeted Degrees and Disciplines:": None,
                                         "Application Documents Required:": None} 
                
                
                tds = tr.find_all('td') # iterating all cells in each row.
                count = 0
                job_info_list = []
                for x in tds[0:-1]: # starting with the 3 buttons
                  
                    
                    if (count == 0): # at 3 buttons
                        link_container = x.find_all('a')[2]
              
                        onclick_link = link_container['onclick']
           
                        # clicking the onclick link:
                        browser.execute_script(onclick_link)
                        browser.switch_to.window(browser.window_handles[1])
                        
                        # give the browser 2 seconds for the website to catch up with the code
                        browser.implicitly_wait(2)
                        
                        time.sleep(3)
                        # do stuff, locate the texts of 'job posting importants'
                        
                        #saving the contents of the html of the new page:
                        indiv_html = browser.execute_script("return document.body.innerHTML")
                        indiv_posting = BeautifulSoup(indiv_html, "html.parser")
                        
                        # finding the third 'table' in the page, as the first table is no use to us
                        job_posting_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[1]
                        print("this is the job posting table")
                        #print(job_posting_information_table)
                       
                        
                        # iterating through all rows in the table
                        for row in job_posting_information_table.find_all('tr'): 
                            cell_list = row.find_all('td')
                            try:
                                key = cell_list[0].get_text().strip()
                            except:
                                continue
                        
        
                            if (key in job_posting_importants):
                                try:
                                    job_posting_importants[key] = cell_list[1].get_text().strip()
                                except:
                                    job_posting_importants[key] = ""
                    

                        application_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[2]
                        job_posting_importants["Application Documents Required:"] = application_information_table.find_all('td')[3]
    
                        browser.close() # close current tab
                        browser.switch_to.window(browser.window_handles[0])
                    
                        #print("lala")
                    if (count >= 2):  
                        #print(x.text)
                        job_info_list.append(str(x.text))
                        
                    count += 1
                        
                    
                for col in job_posting_importants_order:
                    job_info_list.append(job_posting_importants[col])
                
                df.loc[row_count] = job_info_list
                row_count+=1
                
            # navigate to the next page and get the HTML, next page buttons
            browser.find_element_by_link_text("»").click()
            time.sleep(2)
            page_html = browser.execute_script("return document.body.innerHTML")
            next_page_buttons = re.findall(pattern, page_html)


if __name__ == "__main__":
    main()

Are you ready?y
Getting job lists ...
Working
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting 

In [22]:
df


,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,185132,\n\n\n\t\t\t\t\t\t\t\t\t\tAccounting Analyst\n...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",27,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"[\n, [University of Waterloo Co-op Work Histor..."
1,185136,\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity\n\t\t\t\t\t...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",75,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment.\...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"[\n, [University of Waterloo Co-op Work Histor..."
2,186638,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",64,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"[\n, [University of Waterloo Co-op Work Histor..."
3,186639,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",184,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"[\n, [University of Waterloo Co-op Work Histor..."
4,186640,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",41,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: \n\nConfidence: A...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"[\n, [University of Waterloo Co-op Work Histor..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,186738,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,Applied Mind Inc,Divisional Office,1,Open for Applications,Ottawa,"Intermediate, Senior",20,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Applied Mind Inc. is a Canadian start-up compa...,As a Front End Software Developer you will be:...,What we'll require from you...\n- UI/UX design...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"[\n, [University of Waterloo Co-op Work Histor..."
96,186750,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,Applied Mind Inc,Divisional Office,1,Open for Applications,Ottawa,"Intermediate, Senior",20,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Applied Mind Inc. is a Canadian start-up compa...,As an Embedded Software Developer you will be:...,What we'll require from you...\n- Knowledge of...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"[\n, [University of Waterloo Co-op Work Histor..."
97,186281,\n\n\n\n\t\t\t\t\t\t\t\t\t\tSales Support Offi...,ApplyBoard,Divisional Office,1,Open for Applications,Kitchener,"Junior, Intermediate",24,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,About ApplyBoard\nApplyBoard is an online plat...,Key Responsibilities:\n\nRepresent the ApplyBo...,Preferred Qualifications:\n\nComfortable makin...,Targeted Clusters

In [45]:
a = {"bob": 2, 'sam':6, 'asd':123}
a.values()

dict_values([2, 6, 123])

In [31]:
' '.join("j, i, s".split())

'j, i, s'

In [ ]:
e2jin@uwaterloo.ca
Eevee1218.